<a href="https://colab.research.google.com/github/findQin/ColabCode/blob/master/rnn_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [17]:
from scipy.spatial.distance import cdist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
dataset = tfds.load('imdb_reviews/plain_text', as_supervised=True)

In [23]:
train_data, test_data = dataset['train'], dataset['test']

In [25]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [26]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [27]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [28]:
x_train, y_train = next(iter(train_data.batch(25000)))

In [29]:
print("Train-set size: ", len(x_train))

Train-set size:  25000


In [30]:
x_test, y_test = next(iter(test_data.batch(25000)))

In [32]:
print("Test-set size:  ", len(x_test))

Test-set size:   25000


In [41]:
x_train[0]

<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">

In [40]:
x_train[0].numpy()

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [46]:
x_train_text = x_train.numpy().tolist()

In [48]:
type(x_train_text)

list

In [50]:
for x in x_train_text[:10]:
  print(type(x))


<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>


In [53]:
x_train_data = []

for x in x_train_text[:]:
  x_train_data.append(x.decode('utf-8'))

In [54]:
len(x_train_data)

25000

In [61]:
x_test_text = x_test.numpy().tolist()

In [63]:
type(x_test_text)

list

In [64]:
x_test_data = []

for x in x_test_text[:]:
  x_test_data.append(x.decode('utf-8'))

In [34]:
y_train[0]

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [35]:
num_words = 10000

In [37]:
tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")

In [55]:
tokenizer.fit_on_texts(x_train_data)

In [56]:
tokenizer.word_index


{'<OOV>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'on': 21,
 'not': 22,
 'you': 23,
 'are': 24,
 'his': 25,
 'have': 26,
 'he': 27,
 'be': 28,
 'one': 29,
 'all': 30,
 'at': 31,
 'by': 32,
 'an': 33,
 'they': 34,
 'who': 35,
 'so': 36,
 'from': 37,
 'like': 38,
 'her': 39,
 'or': 40,
 'just': 41,
 'about': 42,
 "it's": 43,
 'out': 44,
 'if': 45,
 'has': 46,
 'some': 47,
 'there': 48,
 'what': 49,
 'good': 50,
 'more': 51,
 'when': 52,
 'very': 53,
 'up': 54,
 'no': 55,
 'time': 56,
 'she': 57,
 'even': 58,
 'my': 59,
 'would': 60,
 'which': 61,
 'only': 62,
 'story': 63,
 'really': 64,
 'see': 65,
 'their': 66,
 'had': 67,
 'can': 68,
 'were': 69,
 'me': 70,
 'well': 71,
 'than': 72,
 'we': 73,
 'much': 74,
 'been': 75,
 'bad': 76,
 'get': 77,
 'will': 78,
 'do': 79,
 'also': 80,
 'into': 81,
 'people': 82,
 '

In [57]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_data)

In [58]:
x_train_data[1]

'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'

In [60]:
np.array(x_train_tokens[1])

array([  11,   26,   75,  571,    6,  805, 2354,  313,  106,   19,   12,
          7,  629,  686,    6,    4, 2219,    5,  181,  584,   64, 1454,
        110, 2263,    3, 3951,   21,    2,    1,    3,  258,   41, 4677,
          4,  174,  188,   21,   12, 4078,   11, 1578, 2354,   86,    2,
         20,   14, 1907,    2,  112,  940,   14, 1811, 1340,  548,    3,
        355,  181,  466,    6,  591,   19,   17,   55, 1817,    5,   49,
         14, 4044,   96,   40,  136,   11,  972,   11,  201,   26, 1046,
        171,    5,    2,   20,   19,   11,  294,    2, 2155,    5,   10,
          3,  283,   41,  466,    6,  591,    5,   92,  203,    1,  207,
         99,  145, 4382,   16,  230,  332,   11, 2486,  384,   12,   20,
         31,   30])

In [65]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_data)

In [68]:
x_train_tokens[:2]

[[12,
  14,
  33,
  425,
  392,
  18,
  90,
  28,
  1,
  9,
  32,
  1366,
  3585,
  40,
  486,
  1,
  197,
  24,
  85,
  154,
  19,
  12,
  213,
  329,
  28,
  66,
  247,
  215,
  9,
  477,
  58,
  66,
  85,
  114,
  98,
  22,
  5675,
  12,
  1322,
  643,
  767,
  12,
  18,
  7,
  33,
  400,
  8170,
  176,
  2455,
  416,
  2,
  89,
  1231,
  137,
  69,
  146,
  52,
  2,
  1,
  7577,
  69,
  229,
  66,
  2933,
  16,
  1,
  2904,
  1,
  1,
  1479,
  4940,
  3,
  39,
  3900,
  117,
  1584,
  17,
  3585,
  14,
  162,
  19,
  4,
  1231,
  917,
  7917,
  9,
  4,
  18,
  13,
  14,
  4139,
  5,
  99,
  145,
  1214,
  11,
  242,
  683,
  13,
  48,
  24,
  100,
  38,
  12,
  7181,
  5515,
  38,
  1366,
  1,
  50,
  401,
  11,
  98,
  1197,
  867,
  141,
  10],
 [11,
  26,
  75,
  571,
  6,
  805,
  2354,
  313,
  106,
  19,
  12,
  7,
  629,
  686,
  6,
  4,
  2219,
  5,
  181,
  584,
  64,
  1454,
  110,
  2263,
  3,
  3951,
  21,
  2,
  1,
  3,
  258,
  41,
  4677,
  4,
  174,
  188,
  21,
  1

In [69]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [71]:
num_tokens[:3]

array([117, 112, 132])

In [70]:
np.mean(num_tokens)

235.03316

In [73]:
np.max(num_tokens)

2493

In [74]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

583

In [75]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.94514

In [77]:
pad_type = 'pre'


In [78]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad_type, truncating=pad_type)

In [79]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad_type, truncating=pad_type)

In [80]:
x_train_pad.shape

(25000, 583)

In [81]:
x_test_pad.shape

(25000, 583)

In [82]:
np.array(x_train_tokens[1])

array([  11,   26,   75,  571,    6,  805, 2354,  313,  106,   19,   12,
          7,  629,  686,    6,    4, 2219,    5,  181,  584,   64, 1454,
        110, 2263,    3, 3951,   21,    2,    1,    3,  258,   41, 4677,
          4,  174,  188,   21,   12, 4078,   11, 1578, 2354,   86,    2,
         20,   14, 1907,    2,  112,  940,   14, 1811, 1340,  548,    3,
        355,  181,  466,    6,  591,   19,   17,   55, 1817,    5,   49,
         14, 4044,   96,   40,  136,   11,  972,   11,  201,   26, 1046,
        171,    5,    2,   20,   19,   11,  294,    2, 2155,    5,   10,
          3,  283,   41,  466,    6,  591,    5,   92,  203,    1,  207,
         99,  145, 4382,   16,  230,  332,   11, 2486,  384,   12,   20,
         31,   30])

In [83]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [84]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [85]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [86]:
x_train_data[1]


'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'

In [87]:
tokens_to_string(x_train_tokens[1])

'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all'

In [88]:
embedding_size = 8

In [91]:
model = Sequential()

In [92]:
model.add(Embedding(input_dim=num_words,
          output_dim=embedding_size,
          input_length=max_tokens,
          name='layer_embedding'))


In [93]:
model.add(GRU(units=16, return_sequences=True))

In [94]:
model.add(GRU(units=8, return_sequences=True))

In [95]:
model.add(GRU(units=4))

In [96]:
model.add(Dense(1, activation='sigmoid'))

In [97]:
optimizer = Adam(lr=1e-3)

In [98]:
model.compile(loss='binary_crossentropy',
       optimizer=optimizer,
       metrics=['accuracy'])

In [99]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 583, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 583, 16)           1248      
_________________________________________________________________
gru_1 (GRU)                  (None, 583, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 82,045
Trainable params: 82,045
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Epoch 1/3
372/372 [==============================] - 248s 668ms/step - loss: 0.5005 - accuracy: 0.7471 - val_loss: 0.3660 - val_accuracy: 0.8360
Epoch 2/3
372/372 [==============================] - 250s 671ms/step - loss: 0.3000 - accuracy: 0.8846 - val_loss: 0.3097 - val_accuracy: 0.8728
Epoch 3/3
372/372 [==============================] - 247s 663ms/step - loss: 0.2292 - accuracy: 0.9186 - val_loss: 0.3121 - val_accuracy: 0.8704


In [101]:
result = model.evaluate(x_test_pad, y_test)


782/782 [==============================] - 72s 92ms/step - loss: 0.3440 - accuracy: 0.8584


In [102]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 85.84%
